In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from tqdm import tqdm
import os

accuracies = []

for group_number in range(1, 6):
    
    sdl_class_rdoc_path = f'0.class_document/{group_number}/sdl_class_rdoc.csv'
    test_path = f'../../translation/0.result/{group_number}/test_p.csv'
    
    if not os.path.exists(test_path):
        print(f"test file for Group {group_number} does not exist. Skipping...")
        continue
    
    sdl_class_rdoc_csv = pd.read_csv(sdl_class_rdoc_path, low_memory=False)
    test_csv = pd.read_csv(test_path, low_memory=False)
    
    sdl_class_rdoc_csv['tag_description'] = sdl_class_rdoc_csv['tag_description'].fillna('')
    test_csv['tag_description'] = test_csv['tag_description'].fillna('')
    
    test_csv['c_thing'] = ''
    test_csv['c_property'] = ''
    test_csv['c_score'] = ''
    test_csv['c_duplicate'] = 0
    
    combined_tag_descriptions = sdl_class_rdoc_csv['tag_description'].tolist() + test_csv['tag_description'].tolist()
    
    vectorizer = TfidfVectorizer(
        use_idf=True,  
        token_pattern=r'\S+',
        ngram_range=(1, 1),
    )
    
    vectorizer.fit(combined_tag_descriptions)
    
    sdl_class_rdoc_tfidf_matrix = vectorizer.transform(sdl_class_rdoc_csv['tag_description'])
    test_tfidf_matrix = vectorizer.transform(test_csv['tag_description'])
    
    distance_matrix = pairwise_distances(test_tfidf_matrix, sdl_class_rdoc_tfidf_matrix, metric='cosine')
    
    most_similar_indices = distance_matrix.argmin(axis=1)
    most_similar_scores = 1 - distance_matrix.min(axis=1)
    
    test_csv['c_thing'] = sdl_class_rdoc_csv.iloc[most_similar_indices]['thing'].values
    test_csv['c_property'] = sdl_class_rdoc_csv.iloc[most_similar_indices]['property'].values
    test_csv['c_score'] = most_similar_scores
    
    test_csv['cthing_correct'] = test_csv['thing'] == test_csv['c_thing']
    test_csv['cproperty_correct'] = test_csv['property'] == test_csv['c_property']
    test_csv['ctp_correct'] = test_csv['cthing_correct'] & test_csv['cproperty_correct']
    
    mdm_true_count = len(test_csv[test_csv['MDM'] == True])
    accuracy = (test_csv['ctp_correct'].sum() / mdm_true_count) * 100
    accuracies.append(accuracy)
    
    print(f"Accuracy (MDM=True) for Group {group_number}: {accuracy:.2f}%")
    
    output_path = f'0.class_document/{group_number}/test_p_c.csv'
    test_csv.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    false_positive_rows = test_csv[(test_csv['MDM'] == True) & (test_csv['ctp_correct'] == False)]
    
    fp_output_path = f'0.class_document/{group_number}/fp_class.csv'
    false_positive_rows.to_csv(fp_output_path, index=False, encoding='utf-8-sig')

average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy (MDM=True) across all groups: {average_accuracy:.2f}%")


Accuracy (MDM=True) for Group 1: 73.50%
Accuracy (MDM=True) for Group 2: 78.04%
Accuracy (MDM=True) for Group 3: 81.73%
Accuracy (MDM=True) for Group 4: 79.83%
Accuracy (MDM=True) for Group 5: 81.31%
Average Accuracy (MDM=True) across all groups: 78.88%
